In [ ]:
pip install transformers==3

In [ ]:
pip install seqeval

In [ ]:
pip install pytorch-text-crf

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
from collections import Counter
import re
import time
import random
import functools
from crf.crf import ConditionalRandomField

In [ ]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
    !nvidia-smi

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB
Tue Feb 16 15:56:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    31W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                

In [ ]:
import re
dump_chars = '!"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~’،ـ؟؛«» '

def clean_word(word):
    word = word.translate(str.maketrans({key: None for key in dump_chars}))
    
    #remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    word = re.sub(p_tashkeel,"", word)
    
    return word

a = ['أ','إ','آ']

def load_data(filename):
    f = open(filename, 'r',encoding ='utf-8')
    sents = f.read().split('\n. O\n')
    f.close()
    # tokenize words
    words = [None]*len(sents)
    tokens = [None]*len(sents)
    for i, sent in enumerate(sents):
        sent = sent.split('\n')
        words[i] = []
        tokens[i] = []
        for word in sent:
            line = word.rsplit(' ', 1)
            line[0] = clean_word(line[0])
            if line[1] != 'O':
              line[1] = "B-" + line[1]
            if line [1] not in tag_classes:
              line[1] = 'O'
            if len(line[0]) > 0:
                words[i].append(line[0])
                tokens[i].append(line[1])
                    
                
    return [d for d in words if len(d) > 0], [d for d in tokens if len(d) > 0]

# load data
tag_classes = ['B-Clan', 'O', 'B-Prophet',  'B-Pers', 'B-Date',  'B-Allah', 'B-NatOb', 'B-Loc']

In [ ]:
from sklearn.model_selection import train_test_split

sents, labels = load_data ('CANERCorpus-38maxlen.txt')
train_sents, test_sents, train_labels, test_labels = train_test_split(sents, labels, test_size=0.1, random_state=2018)
train_sents, val_sents, train_labels, val_labels = train_test_split(train_sents, train_labels, test_size=0.1, random_state=2018)

In [ ]:
tag = []

for i , label in enumerate (labels) :
  for j , l in enumerate (label):
    if (l == 'B-Crime'):
      tag.append(sents[i][j])

In [ ]:

tag = set (tag)
print (len (tag))

0


In [ ]:
print (test_sents[0])
print (test_labels[0])

['تعتد', 'عند', 'أهل', 'الزوج', 'أربعة', 'أشهر', 'وعشرة', 'أيام', 'وصية', 'يوصون', 'أن', 'تبقى', 'في', 'دار', 'أهل', 'الزوج', 'إلى', 'تمام', 'السنة', 'متاعا', 'يمتعن', 'متاعا', 'بالسكنى', 'والنفقة', 'في', 'تركته', 'الحول', 'سنة', 'كاملة', 'غير', 'إخراج', 'لا', 'يخرجن', 'جناح', 'إثم', 'جاء', 'الميراث', 'أي']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [ ]:
print('Max train sentence length:',len(max(train_sents, key=len)))
print('Max val sentence length:',len(max(val_sents, key=len)))
print('Max test sentence length:',len(max(test_sents, key=len)))

max_len = max(len(max(train_sents, key=len)),len(max(test_sents, key=len)),len(max(val_sents, key=len)))

Max train sentence length: 54
Max val sentence length: 47
Max test sentence length: 54


In [ ]:
bert_model = 'aubmindlab/bert-base-arabertv01'

In [ ]:
from transformers import BertTokenizer,AutoTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = AutoTokenizer.from_pretrained(bert_model,do_lower_case=False)

Loading BERT tokenizer...


In [ ]:
print (tokenizer.tokenize('الاسلام'))

['الاسلام']


In [ ]:
#------------------------sents processing --------------------------
#-------------------keep all tokens--------------
"""
def encode_sentence(sents,labels):
  tokenized_sents = []
  tokenized_labels = []

  for i , sent in enumerate (sents):
    tokenized_word = []
    tokenized_label = []
    tokenized_word.append("[CLS]")
    tokenized_label.append(0)
    for j , word in enumerate (sent):
      word_tokens = tokenizer.tokenize(word)
      if len(word_tokens) > 0:
        tokenized_word.extend(word_tokens)
        # Use the real label id for the first token of the word, and padding ids for the remaining tokens
        tokenized_label.extend([labels[i][j]] + [0] * (len(word_tokens) - 1))
    tokenized_word.append("[SEP]")
    tokenized_label.append(0)
    special_tokens_count = tokenizer.num_special_tokens_to_add()
    if len(tokenized_word) > max_len - special_tokens_count:
        tokenized_word = tokenized_word[: (max_len - special_tokens_count)]
        tokenized_label = tokenized_label[: (max_len - special_tokens_count)]    
    tokenized_sents.append(tokenized_word)
    tokenized_labels.append(tokenized_label)

  
  return tokenized_sents, tokenized_labels
"""

def encode_sentence(sents,labels):
  tokenized_sents = []
  tokenized_labels = []
  Fasttext_seq = []
  for i , sent in enumerate (sents):
    tokenized_word = []
    tokenized_label = []
    #word_seq = []
    tokenized_word.append("[CLS]")
    tokenized_label.append(0)
    #word_seq.append(0)
    for j , word in enumerate (sent):
      word_tokens = tokenizer.tokenize(word)
      tokenized_word.extend (word_tokens)
      tokenized_label.extend([labels[i][j]] + [0] * (len(word_tokens) - 1))
      """
      if word in embedding.vocab:
        word_seq.extend([embedding.vocab[word].index]+ [0] * (len(word_tokens) - 1))
      else:
        word_seq.extend([embedding.vocab['unk'].index]+ [0] * (len(word_tokens) - 1))
      """
    tokenized_word.append("[SEP]")
    tokenized_label.append(0)
    #word_seq.append(0)
    tokenized_sents.append(tokenized_word)
    tokenized_labels.append(tokenized_label)
    #Fasttext_seq.append(word_seq)

  
  return tokenized_sents, tokenized_labels#, Fasttext_seq

"""
      word_tokens = tokenizer.tokenize(word)
      max = 0
      index = 0 
      for k, t in enumerate (word_tokens):
        if (len (t) > max): 
          max = len (t)
          index = k
      tokenized_word.extend (word_tokens)
      for k, t in enumerate (word_tokens):
        if (k == index): 
          tokenized_label.append(labels[i][j])
        else:
          tokenized_label.append(0)
  """

'\n      word_tokens = tokenizer.tokenize(word)\n      max = 0\n      index = 0 \n      for k, t in enumerate (word_tokens):\n        if (len (t) > max): \n          max = len (t)\n          index = k\n      tokenized_word.extend (word_tokens)\n      for k, t in enumerate (word_tokens):\n        if (k == index): \n          tokenized_label.append(labels[i][j])\n        else:\n          tokenized_label.append(0)\n  '

In [ ]:
"""
tr_tokenized_sents , tr_tokenized_labels , tr_Fasttext_seq= encode_sentence(train_sents,train_labels)
val_tokenized_sents , val_tokenized_labels, val_Fasttext_seq = encode_sentence(val_sents,val_labels)
te_tokenized_sents , te_tokenized_labels, te_Fasttext_seq = encode_sentence(test_sents,test_labels)
"""
tr_tokenized_sents , tr_tokenized_labels = encode_sentence(train_sents,train_labels)
val_tokenized_sents , val_tokenized_labels = encode_sentence(val_sents,val_labels)
te_tokenized_sents , te_tokenized_labels = encode_sentence(test_sents,test_labels)

In [ ]:
print ('tokenization result for sent#1:')
print (train_sents[11])
print (tr_tokenized_sents[11])
#print (tokenized_labels[0])
print (tr_tokenized_labels[11])


#print (tr_Fasttext_seq[11])

tokenization result for sent#1:
['الله', 'عليه', 'وسلم', 'في', 'صلاته', 'أو', 'نقص', 'وهذا', 'الكلام', 'مدرج', 'من', 'إبراهيم', 'وما', 'ذاك', 'ما', 'الذي', 'حدث', 'وهو', 'سؤال', 'من', 'لم', 'يشعر', 'بما', 'وقع', 'منه', 'ولا', 'يقين', 'عنده', 'به', 'ولا', 'غلبة', 'ظن', 'فثنى', 'رجليه', 'عطف', 'رجليه', 'وجلس', 'على']
['[CLS]', 'الله', 'عليه', 'وسلم', 'في', 'صلات', '##ه', 'أو', 'نقص', 'وهذا', 'الكلام', 'مدرج', 'من', 'إبراهيم', 'وما', 'ذاك', 'ما', 'الذي', 'حدث', 'وهو', 'سؤال', 'من', 'لم', 'يشعر', 'بما', 'وقع', 'منه', 'ولا', 'يقين', 'عنده', 'به', 'ولا', 'غلب', '##ة', 'ظن', 'ف', '##ثنى', 'رجليه', 'عط', '##ف', 'رجليه', 'وجلس', 'على', '[SEP]']
[0, 'B-Allah', 'O', 'O', 'O', 'O', 0, 'O', 'O', 'O', 'O', 'O', 'O', 'B-Pers', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 0, 'O', 'O', 0, 'O', 'O', 0, 'O', 'O', 'O', 0]


In [ ]:
print('Max tokinized sentence length:',len(max(tr_tokenized_sents, key=len)))
print('Max tokinized sentence length:',len(max(te_tokenized_sents, key=len)))
print('Max tokinized sentence length:',len(max(val_tokenized_sents, key=len)))

Max tokinized sentence length: 86
Max tokinized sentence length: 92
Max tokinized sentence length: 75


In [ ]:
max_len = max(len(max(tr_tokenized_sents, key=len)),len(max(te_tokenized_sents, key=len)),len(max(val_tokenized_sents, key=len)))

In [ ]:
#---------------get id, mask, and seg -----------------------
def sents_processing (sents):
  inputs_ids = []
  masks = []
  seg_ids = []
  for sent in sents:
    l = max_len-len(sent)
    inputs_ids.append (tokenizer.convert_tokens_to_ids([word for word in sent])+ [tokenizer.pad_token_id]*l)
    masks.append([1 for word in sent]+ [0]*l)
    seg_ids.append([0 for word in sent ]+ [0]*l)

  return torch.tensor(inputs_ids, dtype=torch.long), torch.tensor(masks, dtype=torch.long), torch.tensor(seg_ids, dtype=torch.long)

In [ ]:
train_ids, train_masks, train_seg = sents_processing (tr_tokenized_sents)
val_ids, val_masks, val_seg = sents_processing (val_tokenized_sents)
test_ids, test_masks, test_seg = sents_processing (te_tokenized_sents)

print (train_ids[0])
print (train_masks[0])
print (train_seg[0])

tensor([17028,  2819,   883,  7356,   883,  3887, 11628,  8327,   466, 12199,
          398,  3938,  4773, 21595,  4033,  4107, 21985, 26420, 24602,   192,
         4358,   193,  4369,  1557,   731,  2245,   108, 30114,   660, 34226,
         4055,  5071,  3075,   206,   810,  2075,   908,   883,  2811,  6067,
          437, 14078, 31026, 11052,   631, 17641,   437,  2811,  6067,   437,
         1269,  9461,  2520, 17030, 17029, 17029, 17029, 17029, 17029, 17029,
        17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029,
        17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029,
        17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029, 17029,
        17029, 17029])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:

tag2idx = {t: i+1 for i, t in enumerate(tag_classes)}
tag2idx["PAD"] = 0
idx2tag = {i: w for w, i in tag2idx.items()}
def labels_processing (labels):

  labels_ids = []
  for label in labels:
    l = max_len - len (label)
    labels_ids.append ([tag2idx[tag] if tag != 0 else tag for tag in label] + [0]*l)
  return torch.tensor(labels_ids, dtype=torch.long)

In [ ]:
train_labels = labels_processing (tr_tokenized_labels)
test_labels = labels_processing (te_tokenized_labels)
val_labels = labels_processing (val_tokenized_labels)

print (train_labels[0])

tensor([0, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2,
        2, 2, 2, 0, 2, 8, 2, 2, 2, 2, 0, 2, 0, 0, 4, 4, 4, 4, 2, 4, 2, 4, 4, 4,
        4, 4, 4, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


In [ ]:
print('Max inputs_ids length:',len(max(train_labels, key=len)))

Max inputs_ids length: 92


In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#define a batch size
train_batch_size = 32
test_batch_size = 8

# wrap tensors
#train_data = TensorDataset(train_ids, train_masks, train_seg, train_Fasttext_seq, train_labels)
train_data = TensorDataset(train_ids, train_masks, train_seg, train_labels)

# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)

# dataLoader for train set
train_dataloader = DataLoader(train_data, batch_size=train_batch_size, num_workers=2)

# wrap tensors
#val_data = TensorDataset(val_ids, val_masks, val_seg, val_Fasttext_seq, val_labels)
val_data = TensorDataset(val_ids, val_masks, val_seg, val_labels)

# sampler for sampling the data during training
val_sampler = SequentialSampler(val_data)

# dataLoader for validation set
val_dataloader = DataLoader(val_data, batch_size=test_batch_size, num_workers=1)

# wrap tensors
#test_data = TensorDataset(test_ids, test_masks, test_seg, test_Fasttext_seq, test_labels)
test_data = TensorDataset(test_ids, test_masks, test_seg, test_labels)

# sampler for sampling the data during training
test_sampler = SequentialSampler(test_data)

# dataLoader for validation set
test_dataloader = DataLoader(test_data, batch_size=test_batch_size, num_workers=1)

In [ ]:
from transformers import BertModel, BertConfig, AdamW
 
bert = BertModel.from_pretrained(bert_model)

In [ ]:
class BERTNERTagger(nn.Module):
    def __init__(self,
                 bert):
        
        super().__init__()
        
        self.bert = bert 

        self.gruUnits = 384
      
      # dropout layer
        self.dropout = nn.Dropout(0.2)

      # gru layer 1
        self.bidirectional = True
        self.gru_layers = 1
        self.gruUnits = 200
        self.gru_dropout = 0.1
        self.gru = nn.GRU(768, self.gruUnits,  
                          bidirectional = self.bidirectional, 
                          num_layers = self.gru_layers,
                           dropout = 0.5)
        
        self.fc = nn.Linear(self.gruUnits * 2 if self.bidirectional else self.gruUnits, len(tag2idx))
        #self.fc = nn.Linear(768, len(tag2idx))


      #softmax activation function
        self.softmax = nn.LogSoftmax(dim=1)


        self.attn = nn.MultiheadAttention(
                embed_dim=self.gruUnits * 2,
                num_heads=4,
                dropout=0.5
            )
        
        self.crf = ConditionalRandomField(len(tag2idx),
                            label_encoding="BIO",
                            idx2tag=idx2tag # Index to tag mapping
                            )
        #self.embedding = nn.Embedding.from_pretrained(weights, padding_idx = 0)
        #self.embedding.weight.requires_grad = False
        
    def forward(self, input_id,masks, seg, targets=None):

        pool, seq = self.bert(input_id, attention_mask=masks, token_type_ids=seg)

        pool = self.dropout (pool)

        #fast = self.embedding (Fasttext_seq)
        
        pool , h = self.gru(pool)
        pool = self.dropout (pool)

        #combined = self.dropout (combined)

      # output layer
        out = self.fc(pool)

        best_tag_sequence = self.crf.best_viterbi_tag(out, masks)
        
        
        class_probabilities = out * 0.0
        for i, instance_tags in enumerate(best_tag_sequence):
            for j, tag_id in enumerate(instance_tags[0][0]):
                class_probabilities[i, j, int(tag_id)] = 1        

        output = class_probabilities
        
        
        if targets is not None:
          loss= self.loss_fn(out, targets, masks)
        return output, loss

    def loss_fn(self, logits, target, masks):
        log_likelihood = self.crf(logits, target, masks)
        return -log_likelihood / logits.shape[0]

In [ ]:
# pass the pre-trained BERT to our define architecture
model = BERTNERTagger(bert)

# push the model to GPU
model = model.to(device)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
EPOCHS = 10
LEARNING_RATE = 5e-5
WARMUP_RATIO = 0.1
MAX_GRAD_NORM = 1.0
param_optimizer = list(model.named_parameters())
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
optimizer_parameters = [
    {
        "params": [
            p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.01,
    },
    {
        "params": [
            p for n, p in param_optimizer if any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.0,
    },
]

num_train_steps = int(len(train_sents) / train_batch_size * EPOCHS)
print('Number of training steps: ', num_train_steps)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
#optimizer = AdamW(optimizer_parameters, lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=int(WARMUP_RATIO*num_train_steps), num_training_steps=num_train_steps
)


Number of training steps:  1695


In [ ]:
def loss_fn(output, target, mask, num_labels):
    lfn = nn.CrossEntropyLoss(ignore_index=0)
    active_loss = mask.view(-1) == 1
    active_logits = output.view(-1, num_labels)
    active_labels = torch.where(
        active_loss,
        target.view(-1),
        torch.tensor(lfn.ignore_index).type_as(target)
    )
    loss = lfn(active_logits, active_labels)
    return loss

In [ ]:
from sklearn.metrics import recall_score, precision_score, classification_report, accuracy_score, confusion_matrix, f1_score


def align_predictions(predictions, label_ids):
    preds = np.argmax(predictions, axis=2)

    batch_size, seq_len = preds.shape

    out_label_list = [[] for _ in range(batch_size)]
    preds_list = [[] for _ in range(batch_size)]

    for i in range(batch_size):
        for j in range(seq_len):
            if label_ids[i, j] != 0:
                out_label_list[i].append(idx2tag[label_ids[i][j]])
                preds_list[i].append(idx2tag[preds[i][j]])
    return preds_list, out_label_list

def y2label(zipped, mask=0):
    out_true = []
    out_pred = []
    for zip_i in zipped:
        a, b = tuple(zip_i)
        if a != mask :
            out_true.append(idx2tag[a].replace('B-','').replace('I-',''))
            out_pred.append(idx2tag[b].replace('B-','').replace('I-',''))
    return out_true, out_pred

def compute_metrics(predictions,label_ids):
    print ("predictions:" , predictions.shape)
    print ("label_ids:" , label_ids.shape)
    predictions = np.argmax(predictions, axis=2)
    y_zipped = zip(np.array(label_ids).flat, np.array(predictions).flat)
    preds_list, out_label_list = y2label(y_zipped)
    print(classification_report(out_label_list, preds_list,digits=4))
    return {
        "accuracy_score": accuracy_score(out_label_list, preds_list),
        "precision": precision_score(out_label_list, preds_list, average='macro'),
        "recall": recall_score(out_label_list, preds_list, average='macro'),
        "f1": f1_score(out_label_list, preds_list, average='macro'),
    }

In [ ]:
# function to train the model
def train():
  
  model.train()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save model predictions
  total_preds=[]
  # iterate over batches
  for step,batch in enumerate(train_dataloader):
    
    # progress update after every 50 batches.
    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

    # push the batch to gpu
    batch = [r.to(device) for r in batch]
 
    #sent_id, mask, seg, fast, labels = batch
    sent_id, mask, seg, labels = batch

    # clear previously calculated gradients 
    optimizer.zero_grad()        
    # get model predictions for the current batch
    preds, loss = model(sent_id, mask, seg, labels)
    # compute the loss between actual and predicted values
    # add on to the total loss
    #loss = loss_fn(preds, labels, mask, len(tag2idx))
    total_loss = total_loss + loss.item()

    # backward pass to calculate the gradients
    loss.backward()

    # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    # update parameters
    optimizer.step()
    scheduler.step()
    # model predictions are stored on GPU. So, push it to CPU
    preds=preds.detach().cpu().numpy()
    # append the model predictions
    total_preds.append(preds)

  # compute the training loss of the epoch
  avg_loss = total_loss / len(train_dataloader)
  # predictions are in the form of (no. of batches, size of batch, no. of classes).
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  #returns the loss and predictions
  return avg_loss, total_preds

In [ ]:
# function for evaluating the model
def evaluate(dataloader):
  
  print("\nEvaluating...")
  
  # deactivate dropout layers
  model.eval()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save the model predictions
  total_preds = []
  total_labels = []
  # iterate over batches
  for step,batch in enumerate(dataloader):
    
    # Progress update every 50 batches.
    if step % 50 == 0 and not step == 0:
            
      # Report progress.
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(dataloader)))

    # push the batch to gpu
    batch = [t.to(device) for t in batch]

    sent_id, mask, seg, labels = batch

    # deactivate autograd
    with torch.no_grad():
      
      # model predictions
      preds , loss = model(sent_id, mask, seg, labels)
      # compute the validation loss between actual and predicted values
      #loss = loss_fn(preds, labels, mask, len(tag2idx))
      total_loss = total_loss + loss  
      preds = preds.detach().cpu().numpy()
      labels=labels.detach().cpu().numpy()


      total_preds.append(preds)
      total_labels.append(labels)
  # compute the validation loss of the epoch
  avg_loss = total_loss / len(dataloader) 
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)
  total_labels  = np.concatenate(total_labels, axis=0)

  return avg_loss, total_preds, compute_metrics(total_preds,total_labels)

In [ ]:
epochs = EPOCHS
MODEL_PATH = 'BERT_BGRU_softmax(ANERCorp_+_AQMAR).bin'
# set initial loss to infinite
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]
best_f1 = 0
#for each epoch
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    #train model
    train_loss, _ = train()
   
    #evaluate model
    valid_loss, preds, eval_metrics = evaluate(val_dataloader)
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')
    print (eval_metrics)
    if eval_metrics['f1'] > best_f1:
      torch.save(model, MODEL_PATH)
      best_f1 = eval_metrics['f1']


 Epoch 1 / 10
  Batch    50  of    170.
  Batch   100  of    170.
  Batch   150  of    170.

Evaluating...
  Batch    50  of     76.
predictions: (603, 92, 9)
label_ids: (603, 92)
              precision    recall  f1-score   support

       Allah     0.9876    0.9903    0.9889       722
        Clan     0.8667    0.8125    0.8387        80
        Date     0.8444    0.7037    0.7677        54
         Loc     0.7823    0.8779    0.8273       131
       NatOb     0.5536    0.7045    0.6200        44
           O     0.9962    0.9927    0.9944     18238
        Pers     0.9759    0.9914    0.9836      3471
     Prophet     0.9793    0.9683    0.9738       537

    accuracy                         0.9893     23277
   macro avg     0.8732    0.8802    0.8743     23277
weighted avg     0.9897    0.9893    0.9894     23277


Training Loss: 23.351
Validation Loss: 2.083
{'accuracy_score': 0.9893457060617777, 'precision': 0.8732431771072066, 'recall': 0.8801585959908681, 'f1': 0.874303935712

In [ ]:
#model.load_state_dict(torch.load(MODEL_PATH))
test_loss, preds, test_metrics = evaluate (test_dataloader)
print(f"Test Loss = {test_loss}")
print(test_metrics)


Evaluating...
  Batch    50  of     84.
predictions: (670, 92, 9)
label_ids: (670, 92)
              precision    recall  f1-score   support

       Allah     0.9910    0.9784    0.9847       788
        Clan     0.9029    0.9894    0.9442        94
        Date     0.8667    0.9155    0.8904        71
         Loc     0.9310    0.9153    0.9231       118
       NatOb     0.8571    0.9153    0.8852        59
           O     0.9965    0.9962    0.9964     20254
        Pers     0.9883    0.9901    0.9892      3751
     Prophet     0.9874    0.9791    0.9832       718

    accuracy                         0.9935     25853
   macro avg     0.9401    0.9599    0.9495     25853
weighted avg     0.9936    0.9935    0.9936     25853

Test Loss = 2.1762871742248535
{'accuracy_score': 0.9935404015007929, 'precision': 0.9401191577633068, 'recall': 0.9599102230017789, 'f1': 0.9495494396368983}
